# Vanilla LSTMS, 01052017

needs debugging :(

In [ ]:
from datetime import datetime

import pandas as pd
import numpy as np

from keras.models import Model
from keras.layers import Dense, Dropout, Input, LSTM, Embedding
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, ProgbarLogger, TensorBoard
from keras_tqdm import TQDMNotebookCallback


from utils import load_embeddings, extract_questions_from_dataframe, save_submission

%load_ext autoreload
%autoreload 2

In [50]:
class Config(object):
    VOCABULARY_SIZE = 1193514
    EMBEDDING_DIMENSION = 200
    OFFSET = 3
    OOV_TOKEN = 0  # out of vocabulary
    EOS_TOKEN = 1  # end of sentence
    PAD_TOKEN = 2  # padding to max sentence length
    MAX_SENTENCE_LENGTH = 60
    
    def stamp(self, comment):
        return '{date:%Y%m%d_%H%M}_{comment}'.format(
            date=datetime.now(), comment=comment)

In [52]:
%%time
train_dataframe = pd.read_csv('train.csv')
test_dataframe = pd.read_csv('test.csv')
current_config = Config()

embedding_weights, word2idx = load_embeddings(
    'glove.twitter.27B.200d.txt',
    config=current_config
)

questions_A, questions_B, labels = extract_questions_from_dataframe(
    train_dataframe, 
    config=current_config,
    word2idx=word2idx,
    force_reload=False
)

96463 preprocessed questions loaded from disk
CPU times: user 1min 59s, sys: 7.64 s, total: 2min 6s
Wall time: 2min 15s


In [32]:
questions_A

array([[   2,    2,    2, ...,  124,    0,    1],
       [   2,    2,    2, ...,   38,    0,    1],
       [   2,    2,    2, ...,    0,    0,    1],
       ..., 
       [   2,    2,    2, ...,    0,    0,    1],
       [   2,    2,    2, ..., 3382,    0,    1],
       [   2,    2,    2, ...,    0,    0,    1]], dtype=int32)

In [41]:
shared_lstm_layer = LSTM(
    units=200, 
    return_sequences=False, 
    go_backwards=True, 
    dropout=0.15 + np.random.rand() * 0.25
)
shared_embedding_layer = Embedding(
    input_dim=current_config.VOCABULARY_SIZE + current_config.OFFSET, 
    output_dim=current_config.EMBEDDING_DIMENSION, 
    input_length=current_config.MAX_SENTENCE_LENGTH,
    weights=[embedding_weights],
    trainable=False
)

input_A = Input(shape=(current_config.MAX_SENTENCE_LENGTH,))
embeddings_A = shared_embedding_layer(input_A)
sentence_representation_A = shared_lstm_layer(embeddings_A)
dropout_A = Dropout(0.5)(sentence_representation_A)

input_B = Input(shape=(current_config.MAX_SENTENCE_LENGTH,))
embeddings_B = shared_embedding_layer(input_B)
sentence_representation_B = shared_lstm_layer(embeddings_B)
dropout_B = Dropout(0.5)(sentence_representation_B)

merged_model = concatenate([dropout_A, dropout_B])
predictions = Dense(1, activation='sigmoid')(merged_model)

model = Model(inputs=[input_A, input_B], outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 60)            0                                            
____________________________________________________________________________________________________
input_4 (InputLayer)             (None, 60)            0                                            
____________________________________________________________________________________________________
embedding_2 (Embedding)          (None, 60, 200)       238703400                                    
____________________________________________________________________________________________________
lstm_4 (LSTM)                    (None, 200)           320800                                       
___________________________________________________________________________________________

In [131]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
model_checkpoint = ModelCheckpoint(
    monitor='val_loss',
    filepath=current_config.stamp(comment='1') + '.h5', 
    save_best_only=True, 
    save_weights_only=True
)
progress_bar = ProgbarLogger(count_mode='samples')
tensorboard = TensorBoard(
    log_dir='./logs', 
    histogram_freq=1,
)
tqdm_bar = TQDMNotebookCallback()

In [136]:
%%time

training_logs = model.fit(
    x=[questions_A, questions_B], 
    y=labels, 
    epochs=20, 
    batch_size=1024,
    validation_split=0.1, 
    verbose=1,
    callbacks=[early_stopping, model_checkpoint, progress_bar, tensorboard, tqdm_bar]
)

TypeError: descriptor '_fields' for 'OpDef' objects doesn't apply to 'OpDef' object

In [119]:
model.load_weights('20170501_0326_1.h5')
min(training_logs.history['val_loss'])

NameError: name 'training_logs' is not defined

In [120]:
test_questions_A, test_questions_B = extract_questions_from_dataframe(
    test_dataframe,
    config=current_config,
    word2idx=word2idx,
    prediction_mode=True
)

2345796 preprocessed questions loaded from disk


In [121]:
predictions = model.predict(
    x=[test_questions_A, test_questions_B], 
    batch_size=8192, 
    verbose=1
)

   8192/2345796 [..............................] - ETA: 22905s

  16384/2345796 [..............................] - ETA: 20491s

  24576/2345796 [..............................] - ETA: 20223s

  32768/2345796 [..............................] - ETA: 19913s

  40960/2345796 [..............................] - ETA: 19372s

...

2318336/2345796 [============================>.] - ETA: 176s

2326528/2345796 [============================>.] - ETA: 123s

2334720/2345796 [============================>.] - ETA: 71s 

2342912/2345796 [============================>.] - ETA: 18s

2345796/2345796 [==============================] - 15110s  

In [122]:
save_submission(predictions, current_config)

,is_duplicate
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


In [104]:
!wc -l 20170501_0126_1.csv

  536591 20170501_0126_1.csv
